In [1]:
import pandas as pd
import datetime as dt

## Load and Find Candidate Committees

In [2]:
with open("../data/cn_header_file.csv") as f:
    cn_headers = f.read().split(',')

In [3]:
campaigns_to_commitees = pd.read_csv("../data/cn.txt", sep="|", names=cn_headers)

In [4]:
CAND_NAMES = ["KASICH, JOHN R", "TRUMP, DONALD J", "RUBIO, MARCO", "BUSH, JEB", 'CRUZ, RAFAEL EDWARD "TED"']

In [5]:
campaigns_to_commitees[
    (campaigns_to_commitees["CAND_OFFICE"] == "P") &
    (campaigns_to_commitees["CAND_PTY_AFFILIATION"] == "REP") &
    (campaigns_to_commitees["CAND_ELECTION_YR"] == 2016) &
    (campaigns_to_commitees["CAND_NAME"].isin(CAND_NAMES))
][["CAND_NAME", "CAND_PCC"]]

,CAND_NAME,CAND_PCC
4048,"KASICH, JOHN R",C00581876
4188,"CRUZ, RAFAEL EDWARD ""TED""",C00574624
4248,"RUBIO, MARCO",C00458844
4378,"BUSH, JEB",C00579458
5673,"TRUMP, DONALD J",C00580100


## Load and Clean Individual Donations

The "Contributions by Individuals" master file was downloaded from the [FEC's website](http://www.fec.gov/finance/disclosure/ftpdet.shtml) on April 25, 2016. It contains information about every single itemized individual donation (>$200) for every single 2016 campaign. First we separate out only the donations to the five Republican campaign committees we're interested in analyzing.

In the individual data there is a single name field. The code below splits it up and only selects first and last names in an attempt to remove as much of the variation introduced by the reporting from difference campaign committees as possible.

Finally, FEC campaign filings do not assign any unique identifiers to donors. To be able to distinguish and track individual donors, the code below assigns each contribution a `donor_uid` based on the donor's first name, last name, and ZIP code. This approach could result in an undercount of donors if, for instance, there are two people named John Smith in a same ZIP code — but should result in good approximations for the analyses below.

### Load All Itemized Individual Donations

In [6]:
with open("../data/indiv_header_file.csv") as f:
    ind_headers = f.read().split(',')

In [7]:
dtypes = {
    "NAME": str,
    "ZIP_CODE": str,
    "TRANSACTION_DT": str,
    "FILE_NUM": str,
    "MEMO_CD": str,
    "MEMO_TXT": str
}

In [8]:
donors = pd.read_csv("../data/itcont.txt", sep="|", 
                     names=ind_headers, dtype=dtypes)

In [9]:
donors.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00004606,N,M4,P,15951124869,15,IND,"ARNOLD, ROBERT",MCPHERSON,KS,...,SELF,OPTOMETRIST,03102015,1000,NaN,SA11AI.20747,1002259,NaN,NaN,4041320151241796098
1,C00004606,N,M4,P,15951124869,15,IND,"BICKLE, DON",HAYS,KS,...,RETIRED,RETIRED,03302015,1000,NaN,SA11AI.20772,1002259,NaN,NaN,4041320151241796099
2,C00004606,N,M4,P,15951124869,15,IND,"ROSSMAN, RICHARD",OLATHE,KS,...,CRAWFORD SALES COMPANY,BUSINESSMAN,03302015,250,NaN,SA11AI.20759,1002259,NaN,NaN,4041320151241796100
3,C00452383,N,M4,P,15951124897,15,IND,"LLEWELLYN, CHARLES",FREDERICK,MD,...,NaN,NaN,03112015,500,NaN,SA11AI.25088,1002261,NaN,NaN,4041320151241796102
4,C00452383,N,M4,P,15951124897,15,IND,"TYNES, TIMOTHY MR.",VERO BEACH,FL,...,NaN,NaN,03022015,250,NaN,SA11AI.25074,1002261,NaN,NaN,4041320151241796103


### Separate Out Republican Campaign Comittees

In [10]:
republican_primary_commitees = campaigns_to_commitees[
    (campaigns_to_commitees["CAND_OFFICE"] == "P") &
    (campaigns_to_commitees["CAND_PTY_AFFILIATION"] == "REP") &
    (campaigns_to_commitees["CAND_ELECTION_YR"] == 2016) &
    (campaigns_to_commitees["CAND_NAME"].isin(CAND_NAMES))
]["CAND_PCC"].values

In [11]:
republican_primary_donors = donors[
    donors["CMTE_ID"].isin(republican_primary_commitees) &
    (donors["TRANSACTION_PGI"] == "P")
].copy()

### Clean Up Donation Data

In [12]:
def parse_date(date_string):
    if pd.isnull(date_string):
        return None
    else:
        return dt.datetime.strptime(date_string.strip(), "%m%d%Y")

In [13]:
republican_primary_donors["date"] = republican_primary_donors["TRANSACTION_DT"].apply(parse_date)

In [14]:
def extract_lastfirst(name):
    return " ".join(name.split(" ")[:2])

In [15]:
republican_primary_donors["last_first"] = republican_primary_donors["NAME"].apply(extract_lastfirst)

In [16]:
def make_uid(row):
    if pd.isnull(row["last_first"]) or pd.isnull(row["ZIP_CODE"]):
        return None
    else:
        return "|".join([row["last_first"], row["ZIP_CODE"]])

In [17]:
republican_primary_donors["donor_uid"] = republican_primary_donors.apply(lambda x: make_uid(x), axis=1)

## Analyze the Data

The code below uses `donor_uid` to find the donors who made their first donation to a campaign committee after Jeb Bush and Marco Rubio dropped out of the 2016 Republican presidential primary—Feb. 20, 2016 and March 3, 2016 respectively. It then counts how many of those donations were made by donors who had previously given to the Bush or Rubio campaigns.

In [24]:
def find_overlap(still_alive_com):
    still_alive_donors = republican_primary_donors[
            republican_primary_donors["CMTE_ID"] == still_alive_com
           ].sort_values("date").drop_duplicates(subset=["donor_uid"])
    rubio = republican_primary_donors[
                republican_primary_donors["CMTE_ID"] == "C00458844"
            ].sort_values("date").drop_duplicates(subset=["donor_uid"])
    bush = republican_primary_donors[
                republican_primary_donors["CMTE_ID"] == "C00579458"
           ].sort_values("date").drop_duplicates(subset=["donor_uid"])
    rubio_donors = set(rubio["donor_uid"])
    bush_donors = set(bush["donor_uid"])
    donors_post_bush = set(still_alive_donors[still_alive_donors["date"] > "2016-02-20"]["donor_uid"])
    donors_post_rubio = set(still_alive_donors[still_alive_donors["date"] > "2016-03-15"]["donor_uid"])
    print("Bush to {0}: {1} of {2}\nRubio to {0}: {4} of {3}"\
          .format(still_alive_com,
                  len(donors_post_bush.intersection(bush_donors)),
                  len(donors_post_bush),
                  len(donors_post_rubio),
                  len(donors_post_rubio.intersection(rubio_donors))))

### Donald Trump

In [25]:
find_overlap("C00580100")

Bush to C00580100: 0 of 1181
Rubio to C00580100: 0 of 353


### Ted Cruz

In [26]:
find_overlap("C00574624")

Bush to C00574624: 51 of 5431
Rubio to C00574624: 110 of 1701


### John Kasich

In [27]:
find_overlap("C00581876")

Bush to C00581876: 108 of 3490
Rubio to C00581876: 88 of 1330


---

---

---